# DoubleML Analysis: Water Treatment Impact on E.coli Risk

This notebook implements comprehensive Double Machine Learning analysis with:
- Base models (PLR & IRM) with specified covariates
- Extended models (PLR & IRM) with all covariates
- Subsample analysis by RiskSource categories
- **Multi-learner comparison** (Lasso, CART, Random Forest, XGBoost, and mixed combinations)
- LaTeX table outputs for each analysis

**Note**: This notebook is self-contained - it loads raw data and performs all preprocessing internally.

## Analysis Structure:
1. **Data Loading & Preprocessing** - Load raw MICS data and apply transformations
2. **Helper Functions** - Define DML model runners and table formatters
3. **SomeRiskHome Analysis** - Base, Extended, and Subsample models
4. **VeryHighRiskHome Analysis** - Base, Extended, and Subsample models
5. **Multi-Learner Comparison** - Robustness checks across ML algorithms (following Ahrens et al., 2024)

### 5.5 Key Findings from Multi-Learner Comparison

In [ ]:
# Create RMSE comparison table
def create_rmse_table(plr_results, irm_results, outcome_name):
    """Create table comparing RMSE across learners."""
    
    rows = []
    learners = list(plr_results.keys())
    
    for learner in learners:
        plr = plr_results.get(learner)
        irm = irm_results.get(learner)
        
        row = {'Learner': learner}
        
        if plr is not None:
            row['PLR RMSE$_l$'] = f"{plr['rmse_l']:.4f}" if not np.isnan(plr['rmse_l']) else '-'
            row['PLR RMSE$_m$'] = f"{plr['rmse_m']:.4f}" if not np.isnan(plr['rmse_m']) else '-'
        else:
            row['PLR RMSE$_l$'] = '-'
            row['PLR RMSE$_m$'] = '-'
            
        if irm is not None:
            row['IRM RMSE$_g$'] = f"{irm['rmse_l']:.4f}" if not np.isnan(irm['rmse_l']) else '-'
            row['IRM RMSE$_m$'] = f"{irm['rmse_m']:.4f}" if not np.isnan(irm['rmse_m']) else '-'
        else:
            row['IRM RMSE$_g$'] = '-'
            row['IRM RMSE$_m$'] = '-'
        
        rows.append(row)
    
    df = pd.DataFrame(rows)
    
    print(f"\n{'='*80}")
    print(f"RMSE Comparison: {outcome_name}".center(80))
    print(f"{'='*80}")
    print(df.to_string(index=False))
    print(f"{'='*80}")
    print("RMSE$_l$/RMSE$_g$ = outcome model; RMSE$_m$ = treatment/propensity model")
    print("Lower RMSE indicates better nuisance model fit.\n")
    
    return df

# RMSE tables for both outcomes
rmse_somerisk = create_rmse_table(somerisk_plr_learners, somerisk_irm_learners, "SomeRiskHome")
rmse_veryhigh = create_rmse_table(veryhigh_plr_learners, veryhigh_irm_learners, "VeryHighRiskHome")

### 5.4 RMSE Comparison Across Learners

This section compares nuisance model performance (RMSE) across learners, which helps assess whether treatment effect estimates are sensitive to learner choice.

In [ ]:
# Create LaTeX version of paper-style table
def create_latex_paper_table(somerisk_plr, somerisk_irm, veryhigh_plr, veryhigh_irm, n_controls):
    """Generate LaTeX code for the multi-learner comparison table."""
    
    learners = list(somerisk_plr.keys())
    
    latex_lines = [
        r"\begin{table}[htbp]",
        r"\centering",
        r"\caption{Multi-Learner DML Comparison: Effect of Water Treatment on E.coli Risk}",
        r"\label{tab:multi_learner_comparison}",
        r"\small",
        r"\begin{tabular}{l cccc cccc}",
        r"\toprule",
        r"& \multicolumn{4}{c}{SomeRiskHome} & \multicolumn{4}{c}{VeryHighRiskHome} \\",
        r"\cmidrule(lr){2-5} \cmidrule(lr){6-9}",
        r"Learner & PLR & SE & IRM & SE & PLR & SE & IRM & SE \\",
        r"\midrule",
    ]
    
    for learner in learners:
        sr_plr = somerisk_plr.get(learner)
        sr_irm = somerisk_irm.get(learner)
        vh_plr = veryhigh_plr.get(learner)
        vh_irm = veryhigh_irm.get(learner)
        
        def fmt_coef_latex(result):
            if result is None:
                return '-', '-'
            pval = result['pval']
            sig = '^{***}' if pval < 0.01 else ('^{**}' if pval < 0.05 else ('^{*}' if pval < 0.10 else ''))
            return f"${result['coef']:.4f}{sig}$", f"({result['se']:.4f})"
        
        sr_plr_coef, sr_plr_se = fmt_coef_latex(sr_plr)
        sr_irm_coef, sr_irm_se = fmt_coef_latex(sr_irm)
        vh_plr_coef, vh_plr_se = fmt_coef_latex(vh_plr)
        vh_irm_coef, vh_irm_se = fmt_coef_latex(vh_irm)
        
        # Escape special characters in learner name
        learner_escaped = learner.replace('+', r'\texttt{+}').replace('_', r'\_')
        
        latex_lines.append(
            f"{learner_escaped} & {sr_plr_coef} & {sr_plr_se} & {sr_irm_coef} & {sr_irm_se} & "
            f"{vh_plr_coef} & {vh_plr_se} & {vh_irm_coef} & {vh_irm_se} \\\\"
        )
    
    latex_lines.extend([
        r"\midrule",
        f"No. of controls & \\multicolumn{{8}}{{c}}{{{n_controls}}} \\\\",
        f"N observations & \\multicolumn{{8}}{{c}}{{{somerisk_plr[list(somerisk_plr.keys())[0]]['n_obs']:,}}} \\\\",
        r"\bottomrule",
        r"\end{tabular}",
        r"\begin{tablenotes}",
        r"\small",
        r"\item Note: $^{***}$ p$<$0.01, $^{**}$ p$<$0.05, $^{*}$ p$<$0.10. Standard errors in parentheses.",
        r"\item PLR = Partially Linear Regression, IRM = Interactive Regression Model.",
        r"\item All models use 5-fold cross-fitting with consistent sample splits across learners.",
        r"\item Mixed learners (e.g., Lasso + RF) use different algorithms for outcome (ml\_l/ml\_g) vs. treatment (ml\_m) models.",
        r"\end{tablenotes}",
        r"\end{table}",
    ])
    
    return "\n".join(latex_lines)

# Generate and save LaTeX table
latex_paper_table = create_latex_paper_table(
    somerisk_plr_learners, 
    somerisk_irm_learners,
    veryhigh_plr_learners, 
    veryhigh_irm_learners,
    n_controls=len(extended_covariates)
)

with open('tables/multi_learner_comparison.tex', 'w') as f:
    f.write(latex_paper_table)

print("LaTeX table saved to tables/multi_learner_comparison.tex")

In [ ]:
def create_paper_style_table(somerisk_plr, somerisk_irm, veryhigh_plr, veryhigh_irm, n_controls):
    """
    Create a paper-style comparison table with all learners and both outcomes.
    Format similar to the reference table from IZA Discussion Paper.
    """
    
    learners = list(somerisk_plr.keys())
    
    # Build table data
    table_data = []
    
    for learner in learners:
        sr_plr = somerisk_plr.get(learner)
        sr_irm = somerisk_irm.get(learner)
        vh_plr = veryhigh_plr.get(learner)
        vh_irm = veryhigh_irm.get(learner)
        
        # Helper function for formatting
        def fmt_coef(result):
            if result is None:
                return '-', '-'
            pval = result['pval']
            sig = '***' if pval < 0.01 else ('**' if pval < 0.05 else ('*' if pval < 0.10 else ''))
            return f"{result['coef']:.4f}{sig}", f"({result['se']:.4f})"
        
        def fmt_rmse(result, key='rmse_l'):
            if result is None:
                return '-'
            val = result.get(key, np.nan)
            return f"{val:.4f}" if not np.isnan(val) else '-'
        
        sr_plr_coef, sr_plr_se = fmt_coef(sr_plr)
        sr_irm_coef, sr_irm_se = fmt_coef(sr_irm)
        vh_plr_coef, vh_plr_se = fmt_coef(vh_plr)
        vh_irm_coef, vh_irm_se = fmt_coef(vh_irm)
        
        table_data.append({
            'Learner': learner,
            'SomeRisk PLR': sr_plr_coef,
            'SE': sr_plr_se,
            'SomeRisk IRM': sr_irm_coef,
            'SE.1': sr_irm_se,
            'VeryHigh PLR': vh_plr_coef,
            'SE.2': vh_plr_se,
            'VeryHigh IRM': vh_irm_coef,
            'SE.3': vh_irm_se,
        })
    
    df = pd.DataFrame(table_data)
    
    # Print formatted table
    print("\n" + "="*140)
    print("MULTI-LEARNER DML COMPARISON: Effect of Water Treatment on E.coli Risk".center(140))
    print("="*140)
    print(f"{'Dependent Variable:':^20} {'SomeRiskHome':^55} {'VeryHighRiskHome':^55}")
    print(f"{'':^20} {'PLR':^27} {'IRM':^27} {'PLR':^27} {'IRM':^27}")
    print("-"*140)
    
    for _, row in df.iterrows():
        print(f"{row['Learner']:^20} {row['SomeRisk PLR']:^13} {row['SE']:^13} "
              f"{row['SomeRisk IRM']:^13} {row['SE.1']:^13} "
              f"{row['VeryHigh PLR']:^13} {row['SE.2']:^13} "
              f"{row['VeryHigh IRM']:^13} {row['SE.3']:^13}")
    
    print("-"*140)
    print(f"No. of controls: {n_controls}")
    print(f"N observations: {somerisk_plr[list(somerisk_plr.keys())[0]]['n_obs']:,}")
    print("="*140)
    print("Note: *** p<0.01, ** p<0.05, * p<0.10")
    print("Standard errors in parentheses. All models use 5-fold cross-fitting with consistent sample splits.")
    print()
    
    return df

# Create paper-style summary table
paper_table = create_paper_style_table(
    somerisk_plr_learners, 
    somerisk_irm_learners,
    veryhigh_plr_learners, 
    veryhigh_irm_learners,
    n_controls=len(extended_covariates)
)

### 5.3 Summary Comparison Table (Paper-Style Format)

This table presents results in a format similar to Table 2 in Ahrens et al. (2024), showing treatment effects across multiple ML learners for both outcome variables.

In [ ]:
# Create combined comparison table for VeryHighRiskHome
veryhigh_combined_table = create_combined_comparison_table(
    veryhigh_plr_learners, 
    veryhigh_irm_learners,
    "Multi-Learner Comparison: VeryHighRiskHome (Extended Controls)"
)

# Export to LaTeX
latex_veryhigh_learners = veryhigh_combined_table.to_latex(
    index=False, 
    caption="Multi-Learner Comparison for VeryHighRiskHome (Extended Controls). PLR = Partially Linear Regression, IRM = Interactive Regression Model. RMSE values are cross-validated nuisance model losses.",
    label="tab:veryhigh_learner_comparison",
    escape=False
)

with open('tables/veryhigh_learner_comparison.tex', 'w') as f:
    f.write(latex_veryhigh_learners)

In [ ]:
# Run multi-learner comparison for VeryHighRiskHome with extended controls
print("Running PLR models with multiple learners...")
veryhigh_plr_learners = run_multi_learner_comparison(
    data_complete, 'VeryHighRiskHome', extended_covariates, 'plr'
)

print("\nRunning IRM models with multiple learners...")
veryhigh_irm_learners = run_multi_learner_comparison(
    data_complete, 'VeryHighRiskHome', extended_covariates, 'irm'
)

### 5.2 Multi-Learner Comparison: VeryHighRiskHome (Extended Controls)

In [ ]:
# Create combined comparison table for SomeRiskHome
somerisk_combined_table = create_combined_comparison_table(
    somerisk_plr_learners, 
    somerisk_irm_learners,
    "Multi-Learner Comparison: SomeRiskHome (Extended Controls)"
)

# Export to LaTeX
latex_somerisk_learners = somerisk_combined_table.to_latex(
    index=False, 
    caption="Multi-Learner Comparison for SomeRiskHome (Extended Controls). PLR = Partially Linear Regression, IRM = Interactive Regression Model. RMSE values are cross-validated nuisance model losses.",
    label="tab:somerisk_learner_comparison",
    escape=False
)

with open('tables/somerisk_learner_comparison.tex', 'w') as f:
    f.write(latex_somerisk_learners)

In [ ]:
# Run multi-learner comparison for SomeRiskHome with extended controls
print("Running PLR models with multiple learners...")
somerisk_plr_learners = run_multi_learner_comparison(
    data_complete, 'SomeRiskHome', extended_covariates, 'plr'
)

print("\nRunning IRM models with multiple learners...")
somerisk_irm_learners = run_multi_learner_comparison(
    data_complete, 'SomeRiskHome', extended_covariates, 'irm'
)

### 5.1 Multi-Learner Comparison: SomeRiskHome (Extended Controls)

In [ ]:
def create_learner_comparison_table(results_dict, title, model_type='plr'):
    """
    Create formatted comparison table similar to the reference paper style.
    
    Parameters:
    -----------
    results_dict : dict
        Dictionary with learner results
    title : str
        Table title
    model_type : str
        'plr' or 'irm' (affects column naming)
        
    Returns:
    --------
    DataFrame : Formatted results table
    """
    rows = []
    
    for learner_name, result in results_dict.items():
        if result is None:
            continue
            
        pval = result['pval']
        
        # Format significance stars
        if pval < 0.01:
            sig = '***'
        elif pval < 0.05:
            sig = '**'
        elif pval < 0.10:
            sig = '*'
        else:
            sig = ''
        
        rows.append({
            'Learner': learner_name,
            'Coefficient': f"{result['coef']:.4f}{sig}",
            'Std. Error': f"({result['se']:.4f})",
            '95\\% CI': f"[{result['ci_lower']:.4f}, {result['ci_upper']:.4f}]",
            'RMSE$_l$' if model_type == 'plr' else 'RMSE$_g$': f"{result['rmse_l']:.4f}" if not np.isnan(result['rmse_l']) else '-',
            'RMSE$_m$': f"{result['rmse_m']:.4f}" if not np.isnan(result['rmse_m']) else '-',
        })
    
    df = pd.DataFrame(rows)
    
    print(f"\n{'='*100}")
    print(f"{title.center(100)}")
    print(f"{'='*100}")
    print(df.to_string(index=False))
    print(f"{'='*100}")
    print("Note: *** p<0.01, ** p<0.05, * p<0.10")
    print(f"RMSE$_l$/RMSE$_g$ = outcome model loss; RMSE$_m$ = treatment/propensity model loss\n")
    
    return df

def create_combined_comparison_table(plr_results, irm_results, title):
    """
    Create a combined table showing both PLR and IRM results side by side.
    
    Parameters:
    -----------
    plr_results : dict
        PLR model results
    irm_results : dict
        IRM model results
    title : str
        Table title
        
    Returns:
    --------
    DataFrame : Combined results table
    """
    rows = []
    
    learners = list(plr_results.keys())
    
    for learner in learners:
        plr = plr_results.get(learner)
        irm = irm_results.get(learner)
        
        if plr is None and irm is None:
            continue
        
        # PLR results
        if plr is not None:
            plr_pval = plr['pval']
            plr_sig = '***' if plr_pval < 0.01 else ('**' if plr_pval < 0.05 else ('*' if plr_pval < 0.10 else ''))
            plr_coef = f"{plr['coef']:.4f}{plr_sig}"
            plr_se = f"({plr['se']:.4f})"
            plr_rmse_l = f"{plr['rmse_l']:.4f}" if not np.isnan(plr['rmse_l']) else '-'
            plr_rmse_m = f"{plr['rmse_m']:.4f}" if not np.isnan(plr['rmse_m']) else '-'
        else:
            plr_coef = plr_se = plr_rmse_l = plr_rmse_m = '-'
        
        # IRM results
        if irm is not None:
            irm_pval = irm['pval']
            irm_sig = '***' if irm_pval < 0.01 else ('**' if irm_pval < 0.05 else ('*' if irm_pval < 0.10 else ''))
            irm_coef = f"{irm['coef']:.4f}{irm_sig}"
            irm_se = f"({irm['se']:.4f})"
            irm_rmse_g = f"{irm['rmse_l']:.4f}" if not np.isnan(irm['rmse_l']) else '-'
            irm_rmse_m = f"{irm['rmse_m']:.4f}" if not np.isnan(irm['rmse_m']) else '-'
        else:
            irm_coef = irm_se = irm_rmse_g = irm_rmse_m = '-'
        
        rows.append({
            'Learner': learner,
            'PLR Coef.': plr_coef,
            'PLR SE': plr_se,
            'PLR RMSE$_l$': plr_rmse_l,
            'PLR RMSE$_m$': plr_rmse_m,
            'IRM Coef.': irm_coef,
            'IRM SE': irm_se,
            'IRM RMSE$_g$': irm_rmse_g,
            'IRM RMSE$_m$': irm_rmse_m,
        })
    
    df = pd.DataFrame(rows)
    
    print(f"\n{'='*120}")
    print(f"{title.center(120)}")
    print(f"{'='*120}")
    print(df.to_string(index=False))
    print(f"{'='*120}")
    print("Note: *** p<0.01, ** p<0.05, * p<0.10")
    print("RMSE$_l$/RMSE$_g$ = outcome model loss; RMSE$_m$ = treatment/propensity model loss\n")
    
    return df

print("Table creation functions defined.")

In [ ]:
def run_multi_learner_comparison(data, outcome, covariates, model_type='plr'):
    """
    Run DML with multiple learners using consistent sample splitting.
    
    Parameters:
    -----------
    data : DataFrame
        Complete dataset
    outcome : str
        Outcome variable name
    covariates : list
        List of covariate names
    model_type : str
        'plr' or 'irm'
        
    Returns:
    --------
    dict : Results for all learner configurations
    """
    
    # Create DoubleML data object
    dml_data = DoubleMLData(
        data=data,
        y_col=outcome,
        d_cols=treatment_var,
        x_cols=covariates
    )
    
    # Get learner configurations
    configs = get_learner_configs()
    
    # Initialize first model to get sample splitting
    first_config = list(configs.values())[0]
    if model_type == 'plr':
        first_model = DoubleMLPLR(
            dml_data, 
            ml_l=copy.deepcopy(first_config['ml_l']), 
            ml_m=copy.deepcopy(first_config['ml_m']),
            n_folds=5,
            draw_sample_splitting=True
        )
    else:
        first_model = DoubleMLIRM(
            dml_data, 
            ml_g=copy.deepcopy(first_config['ml_g']), 
            ml_m=copy.deepcopy(first_config['ml_m']),
            n_folds=5,
            draw_sample_splitting=True
        )
    
    # Store sample splitting for reuse
    smpls = first_model.smpls
    
    results = {}
    
    for learner_name, config in configs.items():
        print(f"  Running {learner_name}...", end=" ")
        
        try:
            if model_type == 'plr':
                model = DoubleMLPLR(
                    dml_data, 
                    ml_l=copy.deepcopy(config['ml_l']), 
                    ml_m=copy.deepcopy(config['ml_m']),
                    n_folds=5,
                    draw_sample_splitting=False
                )
            else:
                model = DoubleMLIRM(
                    dml_data, 
                    ml_g=copy.deepcopy(config['ml_g']), 
                    ml_m=copy.deepcopy(config['ml_m']),
                    n_folds=5,
                    draw_sample_splitting=False
                )
            
            # Use same sample splitting
            model.set_sample_splitting(smpls)
            
            # Fit model
            model.fit()
            
            # Extract nuisance losses (RMSE)
            nuisance_loss = model.nuisance_loss
            
            # Get RMSE for outcome and treatment models
            if model_type == 'plr':
                rmse_l = np.sqrt(np.mean(nuisance_loss['ml_l'])) if 'ml_l' in nuisance_loss else np.nan
                rmse_m = np.sqrt(np.mean(nuisance_loss['ml_m'])) if 'ml_m' in nuisance_loss else np.nan
            else:
                rmse_l = np.sqrt(np.mean(nuisance_loss['ml_g'])) if 'ml_g' in nuisance_loss else np.nan
                rmse_m = np.sqrt(np.mean(nuisance_loss['ml_m'])) if 'ml_m' in nuisance_loss else np.nan
            
            results[learner_name] = {
                'model': model,
                'coef': model.coef[0],
                'se': model.se[0],
                'ci_lower': model.confint()['2.5 %'].values[0],
                'ci_upper': model.confint()['97.5 %'].values[0],
                'pval': model.pval[0],
                'rmse_l': rmse_l,
                'rmse_m': rmse_m,
                'n_obs': len(data)
            }
            print("Done")
            
        except Exception as e:
            print(f"Error: {str(e)[:50]}")
            results[learner_name] = None
    
    return results

print("Multi-learner comparison function defined.")

In [ ]:
# Additional imports for multi-learner comparison
from sklearn.linear_model import LogisticRegressionCV, LassoCV
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from doubleml import DoubleMLPLR, DoubleMLIRM
import copy

# Define learner configurations
# For binary outcomes, we use classifiers; for propensity score (treatment), we use classifiers
def get_learner_configs():
    """
    Define learner configurations for DML comparison.
    
    For PLR with binary outcome:
    - ml_l: predicts E[Y|X] (outcome model) - use classifier for binary Y
    - ml_m: predicts E[D|X] (propensity score) - use classifier for binary D
    
    For IRM:
    - ml_g: predicts E[Y|X,D] (outcome model) - use classifier for binary Y
    - ml_m: predicts E[D|X] (propensity score) - use classifier for binary D
    """
    
    configs = {
        # Single learner configurations (same learner for both nuisance functions)
        'Lasso': {
            'ml_l': LogisticRegressionCV(cv=5, penalty='l1', solver='saga', max_iter=1000, random_state=42),
            'ml_m': LogisticRegressionCV(cv=5, penalty='l1', solver='saga', max_iter=1000, random_state=42),
            'ml_g': LogisticRegressionCV(cv=5, penalty='l1', solver='saga', max_iter=1000, random_state=42),
        },
        'CART': {
            'ml_l': DecisionTreeClassifier(max_depth=10, min_samples_leaf=20, random_state=42),
            'ml_m': DecisionTreeClassifier(max_depth=10, min_samples_leaf=20, random_state=42),
            'ml_g': DecisionTreeClassifier(max_depth=10, min_samples_leaf=20, random_state=42),
        },
        'Random Forest': {
            'ml_l': RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_leaf=20, 
                                           n_jobs=-1, random_state=42),
            'ml_m': RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_leaf=20,
                                           n_jobs=-1, random_state=42),
            'ml_g': RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_leaf=20,
                                           n_jobs=-1, random_state=42),
        },
        'XGBoost': {
            'ml_l': XGBClassifier(n_estimators=100, max_depth=5, learning_rate=0.1, 
                                  eval_metric='logloss', random_state=42),
            'ml_m': XGBClassifier(n_estimators=100, max_depth=5, learning_rate=0.1,
                                  eval_metric='logloss', random_state=42),
            'ml_g': XGBClassifier(n_estimators=100, max_depth=5, learning_rate=0.1,
                                  eval_metric='logloss', random_state=42),
        },
        # Mixed learner configurations (different learners for outcome vs treatment)
        'Lasso + RF': {
            'ml_l': LogisticRegressionCV(cv=5, penalty='l1', solver='saga', max_iter=1000, random_state=42),
            'ml_m': RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_leaf=20,
                                           n_jobs=-1, random_state=42),
            'ml_g': LogisticRegressionCV(cv=5, penalty='l1', solver='saga', max_iter=1000, random_state=42),
        },
        'RF + XGBoost': {
            'ml_l': RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_leaf=20,
                                           n_jobs=-1, random_state=42),
            'ml_m': XGBClassifier(n_estimators=100, max_depth=5, learning_rate=0.1,
                                  eval_metric='logloss', random_state=42),
            'ml_g': RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_leaf=20,
                                           n_jobs=-1, random_state=42),
        },
    }
    
    return configs

print("Learner configurations defined.")

# DoubleML Analysis: Water Treatment Impact on E.coli Risk

This notebook implements comprehensive Double Machine Learning analysis with:
- Base models (PLR & IRM) with specified covariates
- Extended models (PLR & IRM) with all covariates
- Subsample analysis by RiskSource categories
- LaTeX table outputs for each analysis

**Note**: This notebook is self-contained - it loads raw data and performs all preprocessing internally.

## Variable Definitions

| Variable | Role | Type | Description |
|----------|------|------|-------------|
| **Outcome Variables (Y)** ||||
| `SomeRiskHome` | Dependent | Binary | E.coli risk indicator (1 = some risk, 0 = no risk) |
| `VeryHighRiskHome` | Dependent | Binary | High E.coli risk indicator (1 = very high risk, 0 = otherwise) |
| **Treatment Variable (T)** ||||
| `water_treatment` | Treatment | Binary | Household treats water before drinking (1 = yes, 0 = no) |
| **Subsample Variable** ||||
| `RiskSource` | Stratification | Categorical | Water source risk level: "No risk", "Moderate to high risk", "Very high risk" |
| **Basic Controls** ||||
| `windex5` | Control | Ordinal (0-4) | Wealth index quintile (0=Poorest, 1=Poor, 2=Middle, 3=Rich, 4=Richest) |
| `helevel` | Control | Ordinal (0-2) | Education level (0=None, 1=Primary, 2=Secondary+) |
| `urban` | Control | Binary | Urban residence (1=Urban, 0=Rural) |
| `wq27_decile` | Control | Ordinal (1-10) | Water quality decile based on E.coli contamination |
| `country_cat_*` | Control | Binary (one-hot) | Country fixed effects (24 countries, reference: Bangladesh) |
| `WS1_g_*` | Control | Binary (one-hot) | Water source type groups (7 types, reference: Delivered water) |
| **Extended Controls - Household Composition** ||||
| `Any_U5` | Control | Binary | Household has children under 5 years |
| `Girls_less_than15` | Control | Binary | Household has girls under 15 years |
| `Boys_15or_less` | Control | Binary | Household has boys 15 years or younger |
| **Extended Controls - Sanitation** ||||
| `improved_latrine` | Control | Binary | Household has improved latrine facility |
| `Flush` | Control | Binary | Household has flush toilet |
| `Pit_latrine` | Control | Binary | Household uses pit latrine |
| `Open_defecation` | Control | Binary | Household practices open defecation |
| **Extended Controls - Water Sources & Services** ||||
| `rainy_season` | Control | Binary | Survey conducted during rainy season |
| `RainandSurfaceWater` | Control | Binary | Main water source is rain/surface water |
| `PurchasedWater` | Control | Binary | Household purchases water |
| `Basic_water_service` | Control | Binary | Household has basic water service level |
| `Limited_water_service` | Control | Binary | Household has limited water service level |
| `Unimproved_water_service` | Control | Binary | Household has unimproved water service |
| `ImprovedWaterSource` | Control | Binary | Water source is classified as improved |
| `PipedWater` | Control | Binary | Household has piped water access |
| `WellandSpringWater` | Control | Binary | Main water source is well/spring |
| `water_carrier_edu` | Control | Ordinal | Education level of person who fetches water (-1=missing/NA) |

**Notes:**
- **Base model** uses: `windex5`, `helevel`, `country_cat_*`, `WS1_g_*`
- **Extended model** uses: All controls listed above
- **Subsample analysis** stratifies by `RiskSource` categories

In [1]:
import pandas as pd
import numpy as np
import optuna
from doubleml import DoubleMLData, DoubleMLPLR, DoubleMLIRM
from xgboost import XGBClassifier
from sklearn.preprocessing import OneHotEncoder
import warnings
warnings.filterwarnings('ignore')

## 1. Load Raw Data and Create Variables

In [2]:
# Load raw data
mics = pd.read_csv("mics.csv", low_memory=False)
print(f"Raw dataset shape: {mics.shape}")

Raw dataset shape: (56721, 785)


### 1.1 Define Variable Groups

In [3]:
# ============================================================
# OUTCOME VARIABLES (Y)
# ============================================================
outcome_vars = ['SomeRiskHome', 'VeryHighRiskHome']

# ============================================================
# TREATMENT VARIABLE (T)
# ============================================================
treatment_var = 'water_treatment'

# ============================================================
# SUBSAMPLE VARIABLE
# ============================================================
subsample_var = 'RiskSource'
risk_categories = ['No risk', 'Moderate to high risk', 'Very high risk']

# ============================================================
# BASIC CONTROLS
# ============================================================
# Ordinal (integer-encoded preserving order)
X_basic_ordinal = ['windex5', 'helevel', 'wq27_decile']

# Categorical (one-hot encoded)
X_basic_categorical = ['country_cat', 'WS1_g']

# Binary
X_basic_binary = ['urban']

# ============================================================
# EXTENDED CONTROLS
# ============================================================
# Binary - Household composition
X_extended_household = ['Any_U5', 'Girls_less_than15', 'Boys_15or_less']

# Binary - Sanitation
X_extended_sanitation = ['improved_latrine', 'Flush', 'Pit_latrine', 'Open_defecation']

# Binary - Season & Water sources
X_extended_water = [
    'rainy_season', 'RainandSurfaceWater', 'PurchasedWater',
    'Basic_water_service', 'Limited_water_service', 'Unimproved_water_service',
    'ImprovedWaterSource', 'PipedWater', 'WellandSpringWater'
]

# Ordinal - Extended
X_extended_ordinal = ['water_carrier_edu']

# Combine all extended binary variables
X_extended_binary = X_extended_household + X_extended_sanitation + X_extended_water

### 1.2 Apply Variable Transformations

In [4]:
# Create urban variable from HH6 column
mics['urban'] = mics['HH6'].str.contains('Urban', case=False, na=False).astype(int)

# Define and apply ordinal mappings
ordinal_mappings = {
    'helevel': {'No education': 0, 'Primary': 1, 'Secondary or higher': 2},
    'windex5': {'Poorest': 0, 'Poor': 1, 'Middle': 2, 'Rich': 3, 'Richest': 4},
}

for col, mapping in ordinal_mappings.items():
    if mics[col].dtype == 'object':
        mics[col] = mics[col].map(mapping).astype('Int64')

# Handle water_carrier_edu (98 = missing -> -1 sentinel)
mics['water_carrier_edu'] = mics['water_carrier_edu'].replace(98, -1).astype('Int64')

print("Ordinal transformations applied.")

Ordinal transformations applied.


In [5]:
# One-hot encode categorical variables
onehot_encoder = OneHotEncoder(sparse_output=False, drop='first')
encoded_cats = onehot_encoder.fit_transform(mics[X_basic_categorical])
encoded_df = pd.DataFrame(
    encoded_cats,
    columns=onehot_encoder.get_feature_names_out(X_basic_categorical)
)

# Concatenate with original data
mics = pd.concat([mics.reset_index(drop=True), encoded_df], axis=1)
mics.drop(X_basic_categorical, axis=1, inplace=True)

# Get encoded column names
encoded_cat_cols = list(onehot_encoder.get_feature_names_out(X_basic_categorical))
print(f"One-hot encoded {len(encoded_cat_cols)} categorical columns.")

One-hot encoded 30 categorical columns.


### 1.3 Assemble Final Covariate Lists

In [6]:
# Basic controls (for main analysis)
base_covariates = (
    X_basic_binary +           # urban
    X_basic_ordinal +          # windex5, helevel, wq27_decile
    encoded_cat_cols           # country_cat_*, WS1_g_*
)

# Extended controls (basic + additional)
extended_covariates = (
    X_basic_binary +           # urban
    X_basic_ordinal +          # windex5, helevel, wq27_decile
    X_extended_binary +        # household, sanitation, water
    X_extended_ordinal +       # water_carrier_edu
    encoded_cat_cols           # country_cat_*, WS1_g_*
)

print(f"Base covariates: {len(base_covariates)} variables")
print(f"Extended covariates: {len(extended_covariates)} variables")

Base covariates: 34 variables
Extended covariates: 51 variables


### 1.4 Prepare Final Dataset

In [7]:
# Select all relevant columns
relevant_cols = outcome_vars + [treatment_var, subsample_var] + extended_covariates
data = mics[relevant_cols].copy()

print(f"Dataset shape before dropping NaN: {data.shape}")
print(f"\nMissing values:\n{data.isnull().sum()[data.isnull().sum() > 0]}")

# Drop rows with NaN values (DoubleML requires complete cases)
data_complete = data.dropna()
print(f"\nComplete cases: {data_complete.shape}")

# Display basic info
data_complete.head()

Dataset shape before dropping NaN: (56721, 55)

Missing values:
Open_defecation             1209
RainandSurfaceWater            3
PurchasedWater                 3
Basic_water_service            2
Limited_water_service          2
Unimproved_water_service       3
PipedWater                     3
WellandSpringWater             3
dtype: int64

Complete cases: (55510, 55)


,SomeRiskHome,VeryHighRiskHome,water_treatment,RiskSource,urban,windex5,helevel,wq27_decile,Any_U5,Girls_less_than15,...,country_cat_Togo,country_cat_Trinidad and Tobago,country_cat_Viet Nam,country_cat_Zimbabwe,WS1_g_Packaged/Bottled water,WS1_g_Piped water,WS1_g_Protected well/spring,WS1_g_Surface/Rain water,WS1_g_Tube/Well/Borehole,WS1_g_Unprotected well/spring
0,1,0,0,Moderate to high risk,0,1,0,7,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,0,0,No risk,0,1,0,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,1,0,Very high risk,0,2,0,8,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,1,0,Very high risk,0,2,0,8,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1,0,0,Moderate to high risk,0,0,0,8,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [8]:
# Display RiskSource distribution
print(f"RiskSource value counts:\n{data_complete['RiskSource'].value_counts()}")

RiskSource value counts:
RiskSource
No risk                  23323
Moderate to high risk    20603
Very high risk           11584
Name: count, dtype: int64


## 2. Helper Functions

In [9]:
def create_hyperparameter_space():
    """Define hyperparameter search space for XGBoost"""
    return {
        'n_estimators': {'low': 50, 'high': 200, 'step': 25},
        'max_depth': {'low': 2, 'high': 6},
        'min_child_weight': {'low': 1, 'high': 10},
        'subsample': {'low': 0.6, 'high': 0.9},
        'colsample_bytree': {'low': 0.6, 'high': 0.9},
        'learning_rate': {'low': 0.01, 'high': 0.1}
    }

def run_doubleml_model(data, outcome, covariates, model_type='plr', n_trials=50):
    """
    Run DoubleML model with hyperparameter tuning
    
    Parameters:
    -----------
    data : DataFrame
        Complete dataset
    outcome : str
        Outcome variable name
    covariates : list
        List of covariate names
    model_type : str
        'plr' or 'irm'
    n_trials : int
        Number of Optuna trials
        
    Returns:
    --------
    dict : Model results
    """
    
    # Create DoubleML data object
    dml_data = DoubleMLData(
        data=data,
        y_col=outcome,
        d_cols=treatment_var,
        x_cols=covariates
    )
    
    # Initialize XGBoost classifiers
    if model_type == 'plr':
        ml_l = XGBClassifier(objective='binary:logistic', eval_metric='logloss', random_state=42)
        ml_m = XGBClassifier(objective='binary:logistic', eval_metric='logloss', random_state=42)
        model = DoubleMLPLR(dml_data, ml_l=ml_l, ml_m=ml_m)
        
        def ml_l_params(trial):
            hp = create_hyperparameter_space()
            return {
                'n_estimators': trial.suggest_int('n_estimators', hp['n_estimators']['low'], 
                                                 hp['n_estimators']['high'], step=hp['n_estimators']['step']),
                'max_depth': trial.suggest_int('max_depth', hp['max_depth']['low'], hp['max_depth']['high']),
                'min_child_weight': trial.suggest_int('min_child_weight', hp['min_child_weight']['low'], 
                                                     hp['min_child_weight']['high']),
                'subsample': trial.suggest_float('subsample', hp['subsample']['low'], hp['subsample']['high']),
                'colsample_bytree': trial.suggest_float('colsample_bytree', hp['colsample_bytree']['low'], 
                                                        hp['colsample_bytree']['high']),
                'learning_rate': trial.suggest_float('learning_rate', hp['learning_rate']['low'], 
                                                    hp['learning_rate']['high'], log=True)
            }
        
        def ml_m_params(trial):
            hp = create_hyperparameter_space()
            return {
                'n_estimators': trial.suggest_int('n_estimators', hp['n_estimators']['low'], 
                                                 hp['n_estimators']['high'], step=hp['n_estimators']['step']),
                'max_depth': trial.suggest_int('max_depth', hp['max_depth']['low'], hp['max_depth']['high']),
                'min_child_weight': trial.suggest_int('min_child_weight', hp['min_child_weight']['low'], 
                                                     hp['min_child_weight']['high']),
                'subsample': trial.suggest_float('subsample', hp['subsample']['low'], hp['subsample']['high']),
                'colsample_bytree': trial.suggest_float('colsample_bytree', hp['colsample_bytree']['low'], 
                                                        hp['colsample_bytree']['high']),
                'learning_rate': trial.suggest_float('learning_rate', hp['learning_rate']['low'], 
                                                    hp['learning_rate']['high'], log=True)
            }
        
        param_space = {'ml_l': ml_l_params, 'ml_m': ml_m_params}
        
    else:  # IRM
        ml_g = XGBClassifier(objective='binary:logistic', eval_metric='logloss', random_state=42)
        ml_m = XGBClassifier(objective='binary:logistic', eval_metric='logloss', random_state=42)
        model = DoubleMLIRM(dml_data, ml_g=ml_g, ml_m=ml_m)
        
        def ml_g_params(trial):
            hp = create_hyperparameter_space()
            return {
                'n_estimators': trial.suggest_int('n_estimators', hp['n_estimators']['low'], 
                                                 hp['n_estimators']['high'], step=hp['n_estimators']['step']),
                'max_depth': trial.suggest_int('max_depth', hp['max_depth']['low'], hp['max_depth']['high']),
                'min_child_weight': trial.suggest_int('min_child_weight', hp['min_child_weight']['low'], 
                                                     hp['min_child_weight']['high']),
                'subsample': trial.suggest_float('subsample', hp['subsample']['low'], hp['subsample']['high']),
                'colsample_bytree': trial.suggest_float('colsample_bytree', hp['colsample_bytree']['low'], 
                                                        hp['colsample_bytree']['high']),
                'learning_rate': trial.suggest_float('learning_rate', hp['learning_rate']['low'], 
                                                    hp['learning_rate']['high'], log=True)
            }
        
        def ml_m_params(trial):
            hp = create_hyperparameter_space()
            return {
                'n_estimators': trial.suggest_int('n_estimators', hp['n_estimators']['low'], 
                                                 hp['n_estimators']['high'], step=hp['n_estimators']['step']),
                'max_depth': trial.suggest_int('max_depth', hp['max_depth']['low'], hp['max_depth']['high']),
                'min_child_weight': trial.suggest_int('min_child_weight', hp['min_child_weight']['low'], 
                                                     hp['min_child_weight']['high']),
                'subsample': trial.suggest_float('subsample', hp['subsample']['low'], hp['subsample']['high']),
                'colsample_bytree': trial.suggest_float('colsample_bytree', hp['colsample_bytree']['low'], 
                                                        hp['colsample_bytree']['high']),
                'learning_rate': trial.suggest_float('learning_rate', hp['learning_rate']['low'], 
                                                    hp['learning_rate']['high'], log=True)
            }
        
        param_space = {'ml_g': ml_g_params, 'ml_m': ml_m_params}
    
    # Optimize hyperparameters
    optuna_settings = {
        'n_jobs_optuna': -1,
        'show_progress_bar': True,
        'verbosity': optuna.logging.WARNING,
        'n_trials': n_trials
    }
    
    model.tune_ml_models(ml_param_space=param_space, optuna_settings=optuna_settings)
    
    # Fit the model
    model.fit()
    
    # Extract results
    summary = model.summary
    
    return {
        'model': model,
        'coef': model.coef[0],
        'se': model.se[0],
        'ci_lower': model.confint()['2.5 %'].values[0],
        'ci_upper': model.confint()['97.5 %'].values[0],
        'pval': model.pval[0],
        'n_obs': len(data),
        'summary': summary
    }

def create_results_table(results_dict, title):
    """
    Create formatted results table
    
    Parameters:
    -----------
    results_dict : dict
        Dictionary with model results
    title : str
        Table title
        
    Returns:
    --------
    DataFrame : Formatted results table
    """
    rows = []
    for key, result in results_dict.items():
        pval = result['pval']
        pval_str = "$< 0.0001$" if round(pval, 4) == 0 else f"{pval:.4f}"
    
        rows.append({
            'Model': key,
            'Coefficient': f"{result['coef']:.4f}",
            'Std. Error': f"{result['se']:.4f}",
            '95\\% CI': f"[{result['ci_lower']:.4f}, {result['ci_upper']:.4f}]",
            'p-value': pval_str,
            'n. obs.': f"{result['n_obs']:,}",
        })

    
    df = pd.DataFrame(rows)
    print(f"\n{'='*80}")
    print(f"{title.center(80)}")
    print(f"{'='*80}")
    print(df.to_string(index=False))
    print(f"{'='*80}\n")
    
    return df

## 3. Analysis: SomeRiskHome

### 3.1 Base Models (PLR & IRM)

In [10]:
# Run base models for SomeRiskHome
somerisk_base_plr = run_doubleml_model(data_complete, 'SomeRiskHome', base_covariates, 'plr')
somerisk_base_irm = run_doubleml_model(data_complete, 'SomeRiskHome', base_covariates, 'irm')

# Create results table
somerisk_base_results = {
    'PLR': somerisk_base_plr,
    'IRM': somerisk_base_irm
}

somerisk_base_table = create_results_table(
    somerisk_base_results, 
    "SomeRiskHome - Base Models"
)

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]


                           SomeRiskHome - Base Models                           
Model Coefficient Std. Error            95\% CI    p-value n. obs.
  PLR     -0.0880     0.0050 [-0.0977, -0.0782] $< 0.0001$  55,510
  IRM     -0.0797     0.0078 [-0.0951, -0.0644] $< 0.0001$  55,510



In [11]:
# Export to LaTeX
latex_base_somerisk = somerisk_base_table.to_latex(index=False, caption="SomeRiskHome - Base Models (PLR and IRM)",  label="tab:somerisk_base", )

# Save to tables/ folder
with open('tables/somerisk_base.tex', 'w') as f:
    f.write(latex_base_somerisk)

### 3.2 Extended Models (PLR & IRM)

In [12]:
# Run extended models for SomeRiskHome
somerisk_ext_plr = run_doubleml_model(data_complete, 'SomeRiskHome', extended_covariates, 'plr')
somerisk_ext_irm = run_doubleml_model(data_complete, 'SomeRiskHome', extended_covariates, 'irm')

# Create results table
somerisk_ext_results = {
    'PLR': somerisk_ext_plr,
    'IRM': somerisk_ext_irm
}

somerisk_ext_table = create_results_table(
    somerisk_ext_results, 
    "SomeRiskHome - Extended Models"
)

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]


                         SomeRiskHome - Extended Models                         
Model Coefficient Std. Error            95\% CI    p-value n. obs.
  PLR     -0.0866     0.0050 [-0.0964, -0.0768] $< 0.0001$  55,510
  IRM     -0.0757     0.0075 [-0.0904, -0.0610] $< 0.0001$  55,510



In [13]:
# Export to LaTeX
latex_ext_somerisk = somerisk_ext_table.to_latex(index=False, caption="SomeRiskHome - Extended Models (PLR and IRM)",  label="tab:somerisk_ext")

# Save to tables/ folder
with open('tables/somerisk_ext.tex', 'w') as f:
    f.write(latex_ext_somerisk)

### 3.3 Subsample Analysis by RiskSource

In [14]:
# Subsample analysis for SomeRiskHome
somerisk_subsample_results = {}

for risk_cat in risk_categories:
    print(f"\n{'='*60}")
    print(f"RiskSource: {risk_cat}")
    print(f"{'='*60}")
    
    # Filter data for this RiskSource category
    subsample_data = data_complete[data_complete['RiskSource'] == risk_cat].copy()
    print(f"Sample size: {len(subsample_data)}")
    
    if len(subsample_data) < 100:
        print(f"Warning: Small sample size for {risk_cat}")
        continue
    
    # Run all 4 models
    base_plr = run_doubleml_model(subsample_data, 'SomeRiskHome', base_covariates, 'plr', n_trials=50)
    base_irm = run_doubleml_model(subsample_data, 'SomeRiskHome', base_covariates, 'irm', n_trials=50)
    ext_plr = run_doubleml_model(subsample_data, 'SomeRiskHome', extended_covariates, 'plr', n_trials=50)
    ext_irm = run_doubleml_model(subsample_data, 'SomeRiskHome', extended_covariates, 'irm', n_trials=50)
    
    somerisk_subsample_results[risk_cat] = {
        'Base PLR': base_plr,
        'Base IRM': base_irm,
        'Extended PLR': ext_plr,
        'Extended IRM': ext_irm
    }
    
    # Create table for this subsample
    subsample_table = create_results_table(
        somerisk_subsample_results[risk_cat],
        f"SomeRiskHome - RiskSource: {risk_cat}"
    )
    
    # Export to LaTeX
    latex_subsample = subsample_table.to_latex(
        index=False, 
        caption=f"SomeRiskHome - RiskSource: {risk_cat}",
        label=f"tab:somerisk_{risk_cat.replace(' ', '_').lower()}"
    )
    
    # Save to tables/ folder
    risk_name = risk_cat.replace(' ', '_').lower()
    filename = f'tables/somerisk_{risk_name}.tex'
    with open(filename, 'w') as f:
        f.write(latex_subsample)


RiskSource: No risk
Sample size: 23323


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]


                   SomeRiskHome - RiskSource: Very high risk                    
       Model Coefficient Std. Error            95\% CI    p-value n. obs.
    Base PLR     -0.0841     0.0069 [-0.0976, -0.0706] $< 0.0001$  11,584
    Base IRM     -0.0688     0.0074 [-0.0834, -0.0542] $< 0.0001$  11,584
Extended PLR     -0.0841     0.0069 [-0.0975, -0.0707] $< 0.0001$  11,584
Extended IRM     -0.0753     0.0107 [-0.0962, -0.0544] $< 0.0001$  11,584



## 4. Analysis: VeryHighRiskHome

### 4.1 Base Models (PLR & IRM)

In [15]:
# Run base models for VeryHighRiskHome
veryhigh_base_plr = run_doubleml_model(data_complete, 'VeryHighRiskHome', base_covariates, 'plr')
veryhigh_base_irm = run_doubleml_model(data_complete, 'VeryHighRiskHome', base_covariates, 'irm')

# Create results table
veryhigh_base_results = {
    'PLR': veryhigh_base_plr,
    'IRM': veryhigh_base_irm
}

veryhigh_base_table = create_results_table(
    veryhigh_base_results, 
    "VeryHighRiskHome - Base Models"
)

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]


                         VeryHighRiskHome - Base Models                         
Model Coefficient Std. Error            95\% CI    p-value n. obs.
  PLR     -0.0648     0.0055 [-0.0755, -0.0541] $< 0.0001$  55,510
  IRM     -0.0681     0.0083 [-0.0844, -0.0518] $< 0.0001$  55,510



In [16]:
# Export to LaTeX
latex_base_veryhigh = veryhigh_base_table.to_latex(index=False, caption="VeryHighRiskHome - Base Models (PLR and IRM)", label="tab:veryhigh_base")

# Save to tables/ folder
with open('tables/veryhigh_base.tex', 'w') as f:
    f.write(latex_base_veryhigh)

### 4.2 Extended Models (PLR & IRM)

In [17]:
# Run extended models for VeryHighRiskHome
veryhigh_ext_plr = run_doubleml_model(data_complete, 'VeryHighRiskHome', extended_covariates, 'plr')
veryhigh_ext_irm = run_doubleml_model(data_complete, 'VeryHighRiskHome', extended_covariates, 'irm')

# Create results table
veryhigh_ext_results = {
    'PLR': veryhigh_ext_plr,
    'IRM': veryhigh_ext_irm
}

veryhigh_ext_table = create_results_table(
    veryhigh_ext_results, 
    "VeryHighRiskHome - Extended Models"
)

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]


                       VeryHighRiskHome - Extended Models                       
Model Coefficient Std. Error            95\% CI    p-value n. obs.
  PLR     -0.0653     0.0055 [-0.0760, -0.0545] $< 0.0001$  55,510
  IRM     -0.0668     0.0069 [-0.0803, -0.0533] $< 0.0001$  55,510



In [18]:
# Export to LaTeX
latex_ext_veryhigh = veryhigh_ext_table.to_latex(index=False, caption="VeryHighRiskHome - Extended Models (PLR and IRM)", label="tab:veryhigh_ext")

# Save to tables/ folder
with open('tables/veryhigh_ext.tex', 'w') as f:
    f.write(latex_ext_veryhigh)

### 4.3 Subsample Analysis by RiskSource

In [19]:
# Subsample analysis for VeryHighRiskHome
print("Running Subsample Analysis for VeryHighRiskHome...\n")

veryhigh_subsample_results = {}

for risk_cat in risk_categories:
    print(f"\n{'='*60}")
    print(f"RiskSource: {risk_cat}")
    print(f"{'='*60}")
    
    # Filter data for this RiskSource category
    subsample_data = data_complete[data_complete['RiskSource'] == risk_cat].copy()
    print(f"Sample size: {len(subsample_data)}")
    
    if len(subsample_data) < 100:
        print(f"Warning: Small sample size for {risk_cat}")
        continue
    
    # Run all 4 models
    base_plr = run_doubleml_model(subsample_data, 'VeryHighRiskHome', base_covariates, 'plr', n_trials=50)
    base_irm = run_doubleml_model(subsample_data, 'VeryHighRiskHome', base_covariates, 'irm', n_trials=50)
    ext_plr = run_doubleml_model(subsample_data, 'VeryHighRiskHome', extended_covariates, 'plr', n_trials=50)
    ext_irm = run_doubleml_model(subsample_data, 'VeryHighRiskHome', extended_covariates, 'irm', n_trials=50)
    
    veryhigh_subsample_results[risk_cat] = {
        'Base PLR': base_plr,
        'Base IRM': base_irm,
        'Extended PLR': ext_plr,
        'Extended IRM': ext_irm
    }
    
    # Create table for this subsample
    subsample_table = create_results_table(
        veryhigh_subsample_results[risk_cat],
        f"VeryHighRiskHome - RiskSource: {risk_cat}"
    )
    
    # Export to LaTeX
    latex_subsample = subsample_table.to_latex(
        index=False, 
        caption=f"VeryHighRiskHome - RiskSource: {risk_cat}",
        label=f"tab:veryhigh_{risk_cat.replace(' ', '_').lower()}"
    )
    
    # Save to tables/ folder
    risk_name = risk_cat.replace(' ', '_').lower()
    filename = f'tables/veryhigh_{risk_name}.tex'
    with open(filename, 'w') as f:
        f.write(latex_subsample)

Running Subsample Analysis for VeryHighRiskHome...


RiskSource: No risk
Sample size: 23323


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]


                 VeryHighRiskHome - RiskSource: Very high risk                  
       Model Coefficient Std. Error            95\% CI    p-value n. obs.
    Base PLR     -0.2065     0.0129 [-0.2318, -0.1811] $< 0.0001$  11,584
    Base IRM     -0.2223     0.0196 [-0.2608, -0.1839] $< 0.0001$  11,584
Extended PLR     -0.2071     0.0130 [-0.2325, -0.1817] $< 0.0001$  11,584
Extended IRM     -0.2139     0.0192 [-0.2515, -0.1763] $< 0.0001$  11,584



## 5. Multi-Learner Comparison and Robustness Analysis

Following best practices in empirical DML research (Chernozhukov et al., 2018; Ahrens et al., 2024), we compare multiple machine learning algorithms to demonstrate robustness of our estimates. This section:

1. **Compares multiple learners**: Lasso, Random Forest, XGBoost, and mixed combinations
2. **Reports nuisance model performance**: RMSE for outcome (ml_l/ml_g) and treatment (ml_m) models
3. **Tests learner mixing**: Different optimal learners for outcome vs. treatment prediction
4. **Uses consistent sample splitting**: All models use the same cross-validation folds for fair comparison

This approach validates DML's "machine learning agnostic" property and avoids cherry-picking a single model.